In [1]:
import pandas as pd
import csv
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

data = []
rating_data = pd.read_csv('./data/rating.csv')
user_data = pd.read_csv('./data/naver_user.csv')

del rating_data['reviewNo']

rating_data.drop_duplicates(keep='first')

user_num = len(rating_data['userId'].unique())
movie_num = len(rating_data['movieId'].unique())
print("유저의 수는 {0} 명이고, \n영화의 수는 {1} 개 입니다.".format(user_num, movie_num))
len(rating_data)

유저의 수는 100 명이고, 
영화의 수는 2697 개 입니다.


5123

In [2]:
df = pd.merge(rating_data, user_data, how='outer', on='userId')
df.head(10)

,userId,rating,movieId,reviewNo
0,airf,2,136900,15772038
1,airf,10,163788,15772038
2,airf,10,174065,15772038
3,nanw,10,154667,15772037
4,nanw,10,136900,15772037
5,nanw,10,145335,15772037
6,nanw,9,120141,15772037
7,nanw,9,54704,15772037
8,zxcv,7,113351,15772036
9,zxcv,10,86507,15772036


In [3]:
top = df.groupby(['userId', 'reviewNo'], as_index=False).count().nlargest(10, 'rating')
top = pd.DataFrame(list(top['reviewNo']), columns=['reviewNo'])
top

,reviewNo
0,15771936
1,15771961
2,15771934
3,15771998
4,15772036
5,15772012
6,15771948
7,15771976
8,15771977
9,15771940


In [4]:
movie = []
for reviewNo in top['reviewNo'] :
    id = df[df['reviewNo']==reviewNo].iloc[1]
    movie.append(id)
    
realdata = pd.DataFrame(movie, columns=['reviewNo', 'movieId', 'rating'])
realdata

,reviewNo,movieId,rating
3412,15771936,145162,3
2180,15771961,161967,10
4119,15771934,163788,7
998,15771998,180399,7
9,15772036,86507,10
621,15772012,158653,10
2913,15771948,172174,10
1930,15771976,180399,7
1825,15771977,157297,1
3202,15771940,52429,9


In [5]:
matrix_ds = df.pivot_table(index='reviewNo', columns='movieId', values='rating')
matrix_ds.head(10)

movieId,10002,10003,10004,10005,10006,10008,10009,10012,10016,10018,...,181409,181410,181411,181414,181419,181711,182348,182360,183132,183877
reviewNo,,,,,,,,,,,,,,,,,,,,,
15771922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15771934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def nearest_neighbor_user(user, topN, simFunc) :
    u1 = matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = {}
    
    for uid, row in matrix_ds.iterrows() :
        interSectionU1 = []
        interSectionU2 = []
        if uid == user: continue
            
        for i in ratedIndex :
            if not math.isnan(row[i]) :
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
        sim = simFunc(interSectionU1, interSectionU2)
        
        if not math.isnan(sim): nn[uid] = sim
            
    return sorted(nn.items(), key=itemgetter(1))[:-(topN+1):-1]

In [7]:
def distance_cosine(a,b):
    return round(1-distance.cosine(a,b), 2)

In [8]:
def distance_euclidean(a,b) :
    return round(1/(1+distance.euclidean(a,b)), 2)

In [9]:
def distance_correlation(a,b) :
    return round(1-distance.correlation(a,b), 2)

In [10]:
#2-a)
print("=Cosine=")

for user in top['reviewNo'] :
    print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(int(user), 3, distance_cosine)))

=Cosine=
User 15771936 neighbors : [(15771972, 1.0), (15771993, 0.97), (15772003, 0.94)]
User 15771961 neighbors : [(15771970, 1.0), (15771966, 1.0), (15771947, 1.0)]
User 15771934 neighbors : [(15772009, 1.0), (15771942, 1.0), (15772030, 0.99)]
User 15771998 neighbors : [(15771970, 1.0), (15771965, 1.0), (15771947, 1.0)]
User 15772036 neighbors : [(15771959, 1.0), (15772000, 0.98), (15771966, 0.98)]
User 15772012 neighbors : [(15772003, 1.0), (15772016, 0.95), (15772000, 0.92)]
User 15771948 neighbors : [(15771966, 1.0), (15771945, 1.0), (15771943, 0.98)]
User 15771976 neighbors : [(15771974, 1.0), (15771966, 1.0), (15771947, 1.0)]
User 15771977 neighbors : [(15771971, 1.0), (15771959, 1.0), (15771983, 0.99)]
User 15771940 neighbors : [(15771993, 1.0), (15771981, 1.0), (15771947, 1.0)]


In [11]:
print("=Euclidean=")

for user in top['reviewNo'] :
    print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(int(user), 3, distance_euclidean)))

=Euclidean=
User 15771936 neighbors : [(15771972, 0.25), (15771993, 0.2), (15772020, 0.12)]
User 15771961 neighbors : [(15772031, 0.33), (15771970, 0.33), (15771954, 0.33)]
User 15771934 neighbors : [(15772030, 0.41), (15772003, 0.31), (15771988, 0.29)]
User 15771998 neighbors : [(15771922, 0.17), (15772030, 0.16), (15772005, 0.15)]
User 15772036 neighbors : [(15771959, 1.0), (15771943, 0.24), (15772022, 0.19)]
User 15772012 neighbors : [(15772016, 0.22), (15772003, 0.15), (15771974, 0.13)]
User 15771948 neighbors : [(15771945, 0.41), (15771943, 0.23), (15771938, 0.15)]
User 15771976 neighbors : [(15771974, 0.31), (15771926, 0.31), (15771966, 0.29)]
User 15771977 neighbors : [(15771971, 1.0), (15771959, 0.5), (15771938, 0.25)]
User 15771940 neighbors : [(15771993, 1.0), (15771947, 1.0), (15771981, 0.33)]


In [12]:
print("=Correlation=")

for user in top['reviewNo'] :
    print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(int(user), 3, distance_correlation)))

=Correlation=


D:\Anaconda\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


User 15771936 neighbors : [(15771993, 1.0), (15772022, 0.69), (15772019, 0.5)]
User 15771961 neighbors : [(15771972, 1.0), (15772015, 0.94), (15771974, 0.93)]
User 15771934 neighbors : [(15771942, 1.0), (15772030, 0.98), (15771947, 0.96)]
User 15771998 neighbors : [(15772005, 0.89), (15771922, 0.88), (15771980, 0.66)]
User 15772036 neighbors : [(15771966, 0.97), (15771943, 0.94), (15771940, 0.6)]
User 15772012 neighbors : [(15772016, 0.94), (15771974, 0.58), (15771969, 0.58)]
User 15771948 neighbors : [(15771943, 0.94), (15771938, 0.73), (15771926, 0.59)]
User 15771976 neighbors : [(15771940, 0.85), (15771974, 0.58), (15771927, 0.53)]
User 15771977 neighbors : [(15771971, 1.0), (15771959, 1.0), (15771998, 0.66)]
User 15771940 neighbors : [(15771981, 0.98), (15771995, 0.97), (15771974, 0.94)]


In [13]:
#2-b)
def predict_rating(userid, nn, simFunc):
    neighbor = nearest_neighbor_user(userid,nn,simFunc)
    neighbor_id = [id for id, sim in neighbor]
    
    neighbor_movie = matrix_ds.loc[neighbor_id].dropna(1,how='all',thresh=1)
    neighbor_dict = (dict(neighbor))
    ret=[]
    
    for movieId, row in neighbor_movie.iteritems():
        jsum,wsum = 0,0
        for v in row.dropna().iteritems():
            sim = neighbor_dict.get(v[0],0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([movieId,wsum/jsum])
        
    return ret

In [14]:
resultcos = []
for i in range(10):
    userId = int(realdata.iloc[i].reviewNo)
    movieId = int(realdata.iloc[i].movieId)
    predict = predict_rating(userId,300,distance_cosine)
    for movie in predict:
        if movieId == movie[0]:
            resultcos.append([int(userId),int(movieId),movie[1]])
result_cos = pd.DataFrame(resultcos,columns=['userId','movieId','rating'])
print("=Cosine=")
result_cos

=Cosine=


,userId,movieId,rating
0,15771936,145162,5.796813
1,15771961,161967,8.595979
2,15771934,163788,9.152855
3,15771998,180399,7.000000
4,15772036,86507,8.567416
5,15771976,180399,7.000000
6,15771977,157297,5.796262


In [15]:
realdata_rating = []

for userId in result_cos['userId']:
    realdata_rating.append(float(realdata[realdata['reviewNo']==userId]['rating']))
    
resultdata_rating = result_cos.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating,resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating,resultdata_rating)
print("\nError Rate(Absolute) : ",error_rate_absol)
print("Erorr Rate(Squared) : ",error_rate_squared)


Error Rate(Absolute) :  1.797504852164316
Erorr Rate(Squared) :  5.6406631699636165


In [16]:
resulteuc = []
for i in range(10):
    userId = int(realdata.iloc[i].reviewNo)
    movieId = int(realdata.iloc[i].movieId)
    predict = predict_rating(userId,300,distance_euclidean)
    for movie in predict:
        if movieId == movie[0]:
            resulteuc.append([int(userId),int(movieId),movie[1]])
result_euc = pd.DataFrame(resulteuc,columns=['userId','movieId','rating'])
print("=Euclidean=")
result_euc

=Euclidean=


,userId,movieId,rating
0,15771936,145162,4.166667
1,15771961,161967,9.247312
2,15771934,163788,8.968750
3,15771998,180399,7.000000
4,15772036,86507,8.125000
5,15771976,180399,7.000000
6,15771977,157297,6.525000


In [17]:
realdata_rating = []

for userId in result_euc['userId']:
    realdata_rating.append(float(realdata[realdata['reviewNo']==userId]['rating']))
    
resultdata_rating = result_euc.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating,resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating,resultdata_rating)
print("\nError Rate(Absolute) : ",error_rate_absol)
print("Erorr Rate(Squared) : ",error_rate_squared)


Error Rate(Absolute) :  1.6125864055299546
Erorr Rate(Squared) :  5.692125308277796


In [18]:
resultcor = []
for i in range(10):
    userId = int(realdata.iloc[i].reviewNo)
    movieId = int(realdata.iloc[i].movieId)
    predict = predict_rating(userId,300,distance_correlation)
    for movie in predict:
        if movieId == movie[0]:
            resultcor.append([int(userId),int(movieId),movie[1]])
result_cor = pd.DataFrame(resultcor,columns=['userId','movieId','rating'])
print("=Correlation=")
result_cor

D:\Anaconda\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


=Correlation=


,userId,movieId,rating
0,15771936,145162,5.390625
1,15771961,161967,10.710280
2,15771934,163788,15.230769
3,15771998,180399,7.000000
4,15772036,86507,11.344828
5,15771976,180399,7.000000
6,15771977,157297,0.963964


In [19]:
realdata_rating = []

for userId in result_cor['userId']:
    realdata_rating.append(float(realdata[realdata['reviewNo']==userId]['rating']))
    
resultdata_rating = result_cor.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating,resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating,resultdata_rating)
print("\nError Rate(Absolute) : ",error_rate_absol)
print("Erorr Rate(Squared) : ",error_rate_squared)


Error Rate(Absolute) :  1.8160768895491308
Erorr Rate(Squared) :  10.825001151824129
